# OP classifier for first 151 pokémon using transfer learning

In this notebook we use Pytorch and transfer learning with pretrained models (ResNet158, MobileNetV2 and MobileNetV3 large). Then, we'll compare all the results and plot them.

In [ ]:
%matplotlib inline

In [ ]:
!pip3 install pip --upgrade
!pip3 install --pre torch torchvision torchaudio -f https://download.pytorch.org/whl/nightly/cu113/torch_nightly.html

Import all necessary modules.

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
!pip install tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs

## 1. Load Data


We will use torchvision and torch.utils.data packages for loading the
data.

We have to train a model to classify Pokémon. There are 151 classes, with an average number of x images per class.
This is a small dataset (if we compare with most popular datasets).
We'll use transfer learning for this task.

### 1.1 Dataset split

The dataset must be already splitted into
- **Train set (`train` folder)**: set of samples used during the learning process and is used to fit the parameters (e.g., weights).
- **Validation set (`val` folder)**: set of samples used to tune the hyperparameters (i.e. the architecture) of the classifier. It should follow the same probability distribution as the training data set. 
- **Test set (`test` folder)**: set of samples used to see the performance of the classifier. It should follow the same probability distribution as the training data set. 

### 1.2 Data Augmentation

Data augmentation is important to:

- prevent overfitting;
- have more samples; 
- prevent a neural network from learning irrelevant patterns, essentially boosting overall performance.

The following transformation are applied to train set.

- random resized crop;
- random horizontal flip;
- random perspective.

Finally, `torchvision.transforms.Normalize()` subtracts the channel mean and divides by the channel standard deviation.

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomPerspective(distortion_scale=0.6, p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

Prepare dataloaders for train, val and test set.

In [ ]:
data_dir = '/tf/notebooks/Downloads/test-ml-net/pokemon-pytorch-nuovo/PokemonData/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val','test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val','test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val','test']}
class_names = image_datasets['train'].classes

Verify if GPU or CPU is used.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'

See if GPU is enabled.

In [ ]:
device

Visualize a few images in a grid to see some samples with data augmentation.

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data and show them as [image - pokémon name].
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

## 2. Training the model

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model of all time

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.

### 2.1 Stochastic Gradient Descent

Stochastic gradient descent (SGD) is an optimization algorithm that estimates the error gradient for the current state of the model using examples from the training dataset, then updates the weights of the model using the back-propagation of errors algorithm, referred to as simply backpropagation.

### 2.2 Learning rate and the importance of scheduling the learning rate.

The amount that the weights are updated during training is referred to as the step size or the _learning rate._

The learning rate controls how quickly the model is adapted to the problem. Smaller learning rates require more training epochs given the smaller changes made to the weights each update, whereas larger learning rates result in rapid changes and require fewer training epochs

A learning rate that is too large can cause the model to converge too quickly to a suboptimal solution, whereas a learning rate that is too small can cause the process to get stuck.


### 2.3 Learning rate scheduler

A Learning rate scheduler is a predefined framework that adjusts the learning rate between epochs or iterations as the training progresses. We'll use _learning rate decay_: we select an initial learning rate, then gradually reduce it in accordance with a scheduler.

In [2]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            writer.add_scalar("epoch_loss", epoch_loss, epoch)
            writer.add_scalar("epoch_acc", epoch_acc, epoch)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
                              
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()
 
    writer.flush()
    writer.close()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

See all Pokémon names.

In [ ]:
class_names

## 3. Finetuning the model

Load a pretrained model and reset final fully connected layer (`nn.Linear`).

In [ ]:
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features

# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

############## MobileNetV2 ##################
#model_ft = models.mobilenet_v2(pretrained=True)
#model_ft.classifier = nn.Linear(1280, 151)
#############################################

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Finally, train model.



In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=15)

## 4. Visualizing the model predictions

Generic function to display predictions for a few images from test set.

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model_ft)
plt.ioff()
plt.show()

### 4.1 Obtain test loss and test accuracy.

In [ ]:
eval_losses=[]
eval_accu=[]

model_ft.eval()
running_loss = 0
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in dataloaders['test']:
        images, labels = data[0].to(device),data[1].to(device)
        # calculate outputs by running images through the network
        outputs = model_ft(images)
        loss= criterion(outputs,labels)
        running_loss+=loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
   
    test_loss=running_loss/len(dataloaders['test'])
    accu=100.*correct/total

    eval_losses.append(test_loss)
    eval_accu.append(accu)

    print('Test Loss: %.3f | Accuracy: %.3f'%(test_loss,accu)) 

Save all model weights.

In [ ]:
torch.save(model_ft.state_dict(), "/tf/notebooks/Downloads/test-ml-net/pokemon-pytorch-nuovo/nuovomobilenetv3")

Create new model (pls) and load all trained weights and bias on CPU. Fix, onnx export doesn't like GPUs as model source.

In [ ]:
pls = models.mobilenet_v3_large(pretrained=False)
#pls2 = pls.fc.in_features
#pls.classifier = nn.Linear(1280, 151)
pls.load_state_dict(torch.load("/tf/notebooks/Downloads/test-ml-net/pokemon-pytorch-nuovo/nuovomobilenetv3",map_location=torch.device('cpu')))

In [ ]:
pls.eval()
dummy_input = torch.randn(1, 3, 224, 224)
input_names = [ "actual_input" ]
output_names = [ "output" ]
torch.onnx.export(pls,
                  dummy_input,
                  "/tf/notebooks/Downloads/test-ml-net/pokemon-pytorch-nuovo/nuovomobilenetv3.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names,
                  export_params=True,
                  )

In [ ]:
!pip3 install onnx-tf

Convert to tensorflow model (`pb`).

In [ ]:
!onnx-tf convert -i /tf/notebooks/Downloads/test-ml-net/pokemon-pytorch-nuovo/nuovomobilenetv3.onnx -o /tf/notebooks/Downloads/test-ml-net/pokemon-pytorch-nuovo/nuovomobilenetv3.pb

Convert model to `.tflite`.

In [ ]:
!tflite_convert --saved_model_dir=/tf/notebooks/Downloads/test-ml-net/pokemon-pytorch-nuovo/nuovomobilenetv3.pb --output_file=/tf/notebooks/Downloads/test-ml-net/pokemon-pytorch-nuovo/nuovomobilenetv3.tflite

## Predict images

In [146]:
import torch
from torchvision import transforms, models
import numpy as np
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import torch.nn as nn
device = torch.device('cpu')
#mobilenet = models.mobilenet_v2(pretrained=False)
#mobilenet.classifier = nn.Linear(1280, 151)
#model = torch.load('./nuovomobilenetv2', map_location=device)
#mobilenet.load_state_dict(model)
#mobilenet.eval()
model_ft = models.resnet152(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 151)
model = torch.load('./nuovoresnet152', map_location=device)
model_ft.load_state_dict(model)
model_ft.eval()

test_transforms = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                    ])

In [232]:
pic = './13.jpg'
pic = Image.open(pic)
image_tensor = test_transforms(pic).float()
image_tensor = image_tensor.unsqueeze_(0)

In [233]:
output = model_ft(image_tensor)

In [234]:
sm = nn.Softmax(dim = 1)
probs = sm(output)
print(probs)

tensor([[1.7383e-03, 8.2524e-04, 6.5247e-05, 3.5071e-04, 5.9408e-05, 6.6976e-06,
         1.4547e-04, 3.2106e-04, 2.8036e-04, 7.4395e-06, 2.9445e-05, 3.3467e-05,
         2.2372e-04, 6.6766e-02, 1.2478e-01, 9.8978e-02, 1.7174e-03, 2.3714e-04,
         3.9810e-05, 1.9131e-02, 4.8765e-04, 1.1620e-04, 7.3980e-05, 8.7539e-05,
         7.3875e-05, 1.7003e-04, 2.5404e-02, 3.0819e-05, 3.4128e-04, 1.2524e-04,
         8.3029e-04, 7.1679e-05, 2.6630e-05, 3.0326e-05, 3.7306e-04, 2.8351e-04,
         8.6665e-04, 3.2153e-04, 2.3990e-04, 2.4375e-05, 7.4555e-05, 6.5681e-04,
         7.5601e-05, 1.8700e-05, 8.1081e-04, 3.6523e-04, 4.3538e-04, 2.7829e-05,
         2.6076e-04, 3.6693e-04, 3.1814e-04, 1.1249e-04, 1.2006e-03, 7.2104e-03,
         3.0793e-05, 6.2694e-04, 1.2077e-05, 3.3869e-04, 1.4274e-05, 2.9443e-05,
         1.7436e-05, 2.4287e-03, 4.6200e-04, 6.1250e-05, 2.0974e-02, 2.4703e-03,
         1.2571e-05, 2.2984e-02, 1.1872e-03, 2.8446e-03, 1.8476e-04, 4.6945e-05,
         5.1476e-03, 3.4747e

In [235]:
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('./best_classes.npy')

In [236]:
(top_k_scores, top_k_idx) = torch.topk(probs, 5)
top_k_scores = top_k_scores.detach().numpy()
top_k_scores = np.squeeze(top_k_scores, axis=0)
top_k_idx = np.squeeze(top_k_idx.numpy(), axis=0)
top_k_labels = label_encoder.inverse_transform(top_k_idx)

print(top_k_labels, top_k_idx)

['Mew' 'Charmander' 'Mewtwo' 'Charmeleon' 'Marowak'] [81 14 82 15 78]


In [124]:
test = torch.max(probs)
test

tensor(0.6879, grad_fn=<MaxBackward1>)